<a href="https://colab.research.google.com/github/VIOLDAVE/my_project/blob/main/Massive_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project requirement: https://docs.google.com/document/d/1GkK28wOyjTPZEZuOumKPU0z1NzVT_9sxETjPOcLPVYo/edit?tab=t.0#heading=h.qifoo7co6qtd

Professor : https://malchiodi.di.unimi.it/teaching/AMD-DSE/2024-25/en

Connecting the notebook to my_project directory in GitHub.

In [1]:
import os

if not os.path.exists("my_project"):
    !git clone https://github.com/VIOLDAVE/my_project.git

if os.path.exists("my_project"):
    os.chdir("my_project")
else:
    raise FileNotFoundError("Failed to find or clone the 'my_project' directory.")

Cloning into 'my_project'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 50 (delta 27), reused 17 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 37.63 KiB | 3.42 MiB/s, done.
Resolving deltas: 100% (27/27), done.


Install packages.

In [2]:
!pip install -r requirement.txt

 kaggle  set up and Authentication

In [3]:
import json, os
from google.colab import files

# Upload the file
uploaded = files.upload()

# Process the uploaded file
for fn in uploaded.keys():
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')



    # Move and rename the uploaded file to the correct kaggle.json path
    os.makedirs("/root/.kaggle", exist_ok=True)
    os.rename(fn, "/root/.kaggle/kaggle.json")
    os.chmod("/root/.kaggle/kaggle.json", 0o600)


Saving kaggle2.json to kaggle2.json
User uploaded file "kaggle2.json" with length 66 bytes


dowloading files

In [4]:
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 99% 1.05G/1.06G [00:12<00:00, 259MB/s]
100% 1.06G/1.06G [00:14<00:00, 78.9MB/s]


unzipping the dataset

In [5]:
import zipfile
import os

zip_file_path = "/content/my_project/amazon-books-reviews.zip"
extract_path = "/content/data"

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

# List the extracted files to confirm
!ls /content/data

Files extracted to: /content/data
books_data.csv	Books_rating.csv


Loading the dataset on Spark

In [6]:
# Load the dataset with spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()
books_rating = spark.read.csv("/content/data/Books_rating.csv", header=True, inferSchema=True)

In [7]:
books_rating.count()

3000000

In [8]:
books_rating.summary()

DataFrame[summary: string, Id: string, Title: string, Price: string, User_id: string, profileName: string, review/helpfulness: string, review/score: string, review/time: string, review/summary: string, review/text: string]

Show a few raws of the dataset

In [9]:
books_rating.show(5)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| NULL|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| NULL|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

Creating a sample of dataset becase the file is too heavy for my colab space

In [10]:
sample = 0.05 # can be adjusted as needed

In [11]:
books_sample = books_rating.sample(fraction=sample, seed=42)
books_sample.show(10)
size = books_sample.count()

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|0595344550|Whispers of the W...|10.95| AUR0VA5H0C66C|"LoveToRead ""Act...|               1/2|         1.0| 1119225600|        Buyer beware|"This is a self-p...|
|B000879GGE|A husband for Kutani| NULL|A2HDZHLMT3L5IO|"Laurence Bush ""...|               0/0|         5.0| 1141603200|Unique Weird Orie...|"Exotic tales of ...|
|0553763121|Overcoming Hypert...|23.00|A3TQFPF0QGVSJV|   "Woody ""Woody"""|               2/2|         5.0| 1352073600|A must read for a...|"Books by Dr Kenn...|
|B000NKGYMK|    Alaska Sourd

In [12]:
size

150324

Checking null values. I will use the review column which has strings.

In [13]:
# Checking  for null values in the 'review/text' column
null_df = books_rating.filter(
    books_rating['review/text'].isNull() | (books_rating['review/text'].rlike(r'^\s*$'))
)

null_df.show(15)

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|        Id|               Title|               Price|             User_id|         profileName|  review/helpfulness|        review/score|         review/time|      review/summary|review/text|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|B00085T7O2|"""Catch 'em aliv...|",,ASEH0CVYVGZCZ,...|                 0/1|                 5.0|          1341705600|Fascinating life ...|There are many id...|                NULL|       NULL|
|B00085T7O2|"""Catch 'em aliv...|",,A91QQ7IYBAK2Q,...| terrible forward...|I found the book ...|                NULL|                NULL|                NULL|                NULL|       NULL|
|B0007DMFZS|"""Beloved friend...|",

There are copies of the'review/text' null entries.

 Aside from the Title and the IDs, they have the same columns.  A qualitative examination reveals that they are about the same book even though the titles are different.

There are numerous other reviews of Lord of the Rings.  I believe that the NA values should be retained because they cannot be discarded.  The title can also be used as a similarity check.

In [14]:
from pyspark.sql.functions import col
filter_df = books_rating.filter(
    col("Title").contains("Wonderful Worship in Smaller Churches")
).select("Title", "review/text").limit(100)

# result
filter_df.show(100, truncate=True)

+--------------------+--------------------+
|               Title|         review/text|
+--------------------+--------------------+
|Wonderful Worship...|I just finished t...|
|Wonderful Worship...|"Many small churc...|
|Wonderful Worship...|I just finished r...|
|Wonderful Worship...|"I hadn't been a ...|
+--------------------+--------------------+



Null values in subsample books.

In [15]:
# Check for null values in the 'review/text' column
Subsample_book = books_sample.filter(
    books_sample['review/text'].isNull() | (books_sample['review/text'].rlike(r'^\s*$'))
)
#Display sample books
Subsample_book.show(35)

+----------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+-----------+
|        Id|               Title|               Price|             User_id|         profileName|review/helpfulness|        review/score|         review/time|      review/summary|review/text|
+----------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+-----------+
|B000I1VJLA|The Lord of the R...|                NULL|                NULL|                NULL|               0/1|                 5.0|           938563200|have only one wor...|       NULL|
|B00085H0VE|"""Charge that to...|"" and other Gosp...|",,A2AKETSGUW9KYI...|                 1/1|               5.0|          1211760000|One of the best p...|This little book ...|       NULL|
|B0006AQJN6|"""A giant in the...|"": A biogra

#Jaccard Similarity


#Tokenisation
 To test the Jaccard Similarity technique, I will utilise a subsample of the dataset because it is too large.  After that, I'll attempt to top scale it to the entire dataset.  Additionally, I'll parallelise the Jaccard Similarity calculation using the MapReduce technique.

## Apply tokenization

### Subtask:
Apply the `RegexTokenizer` (to break down the text into individual words or tokens.) and create new column the `review/text`  of  Spark DataFrame to get tokens.


**Meaning**
It applies the pre-defined RegexTokenizer to the books_sample DataFrame and displays the schema of the resulting DataFrame to confirm the new column.



In [16]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import RegexTokenizer
spark = SparkSession.builder.appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()
tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern=r"\W")
tokenized_df = tokenizer.transform(books_sample)
tokenized_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



## Remove stop words

### Subtask:
Use `StopWordsRemover` to remove common English stop words from the tokenized reviews.


**Reasoning**:
Use StopWordsRemover to remove common English stop words from the tokenized reviews.



In [17]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="reviews/tokens", outputCol="reviews/filtered_tokens")
filtered_df = remover.transform(tokenized_df)
filtered_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- reviews/filtered_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



## Calculate jaccard similarity

### Subtask:
Implementing a method to calculate the Jaccard similarity between the processed review tokens for pairs of books.



 Python function to calculate Jaccard similarity and register it as a Spark UDF. Then, join the DataFrame with itself to create pairs of books and apply the UDF to calculate the similarity between their filtered tokens.



In [18]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

def jaccard_similarity(list1, list2):
    if not list1 or not list2:
        return 0.0
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0.0

jaccard_udf = udf(jaccard_similarity, DoubleType())

# Create pairs of books for similarity calculation
book_pairs = filtered_df.alias("df1").join(
    filtered_df.alias("df2"),
    col("df1.Id") < col("df2.Id")
)

# Calculate Jaccard similarity for each pair
similarity_df = book_pairs.withColumn(
    "jaccard_similarity",
    jaccard_udf(col("df1.reviews/filtered_tokens"), col("df2.reviews/filtered_tokens"))
)

# Select relevant columns and show the results
similarity_df.select(
    col("df1.Id").alias("book1_id"),
    col("df2.Id").alias("book2_id"),
    col("jaccard_similarity")
).show(20)

+----------+----------+--------------------+
|  book1_id|  book2_id|  jaccard_similarity|
+----------+----------+--------------------+
|0595344550|B000879GGE|                 0.0|
|0595344550|B000NKGYMK|0.056338028169014086|
|0595344550|B000NKGYMK|0.013513513513513514|
|0595344550|0789480662| 0.07407407407407407|
|0595344550|0789480662|0.012048192771084338|
|0595344550|157067051X|0.015151515151515152|
|0595344550|B0007DVHU2|0.009523809523809525|
|0595344550|0781810698|0.013888888888888888|
|0595344550|B0006DWYDW|0.015384615384615385|
|0595344550|3526448353|0.034482758620689655|
|0595344550|B0006D6DRK|  0.0425531914893617|
|0595344550|0934638160|0.013333333333333334|
|0595344550|0934638160|0.012987012987012988|
|0595344550|B000FZEKVA|0.018691588785046728|
|0595344550|1861263678|0.030303030303030304|
|0595344550|0671551345|0.029850746268656716|
|0595344550|0671551345|0.011363636363636364|
|0595344550|0671551345|0.029850746268656716|
|0595344550|B000N7612G| 0.02702702702702703|
|059534455

Calculating Jaccard similarity

In [19]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

In [20]:
tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern=r"\W") # \W is a regex pattern that matches any non-word character

The text in the reviews is tokenized into individual words using Spark's `RegexTokenizer`. This process is a foundational step for many text analysis tasks. Using Spark for tokenization leverages its distributed processing capabilities, which can be faster for large datasets compared to certain methods, including some neural network-based approaches that might require different preprocessing or computational resources.

While effective for breaking down text, this tokenization method has limitations, such as not explicitly handling compound terms (like "well-being") as single units. However, for the initial strategy of calculating Jaccard similarity based on individual word sets, this approach is a suitable starting point.

In [21]:
stopwords_remover = StopWordsRemover(inputCol="reviews/tokens", outputCol="tokens/nostopwords")

The `StopWordsRemover` uses a predefined list of common English stop words (like "the," "is," "in") and removes them from the tokenized reviews. This step is crucial because these common words appear frequently in most texts but carry little specific meaning. For methods like Jaccard Similarity, which rely on the overlap of unique terms between documents, including stop words would inflate the similarity scores based on irrelevant terms, skewing the results. Removing them helps to focus on the more meaningful words that truly contribute to the content's uniqueness and similarity.

In [22]:
def remove_numbers(tokens):
    return [token for token in tokens if not token.isdigit()]

Frequently, the meaning of a review is not affected by numbers.  We can eliminate the numbers from the text because the score in our dataset is kept in a distinct column.

In [23]:
remove_numbers_udf = udf(remove_numbers, ArrayType(StringType())) # Specify the return type as ArrayType(StringType), meaning an array/list of strings


Because Sparks is not a Python object, it must be used in the pipeline as a user-defined function (UDF).

In [24]:
tokenized_books_sample = tokenizer.transform(books_sample) # Apply the tokenizer to the dataset
tokenized_reviews_nostopwords_sample = stopwords_remover.transform(tokenized_books_sample) # Remove the stop words from the tokenized dataset
tokenized_rw_nsw_nn_sample = tokenized_reviews_nostopwords_sample.withColumn("tokens/nostopwords/nonumbers", remove_numbers_udf(col("tokens/nostopwords"))) # Remove the numbers from the dataset



In [25]:

books_sample.show(10)
tokenized_rw_nsw_nn_sample.show(10, truncate=True)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|0595344550|Whispers of the W...|10.95| AUR0VA5H0C66C|"LoveToRead ""Act...|               1/2|         1.0| 1119225600|        Buyer beware|"This is a self-p...|
|B000879GGE|A husband for Kutani| NULL|A2HDZHLMT3L5IO|"Laurence Bush ""...|               0/0|         5.0| 1141603200|Unique Weird Orie...|"Exotic tales of ...|
|0553763121|Overcoming Hypert...|23.00|A3TQFPF0QGVSJV|   "Woody ""Woody"""|               2/2|         5.0| 1352073600|A must read for a...|"Books by Dr Kenn...|
|B000NKGYMK|    Alaska Sourd

To prepare the tokens for further analysis and to create a representation of the text data, I used `CountVectorizer`. This tool builds a vocabulary of the most frequent tokens in the dataset.

*   **`vocabSize`**: This parameter sets the maximum number of unique tokens to include in the vocabulary. By limiting the vocabulary size, we focus on the most relevant terms and manage the dimensionality of the data.
*   **`minDF`**: This parameter specifies the minimum number of documents a token must appear in to be considered for the vocabulary. This is important for filtering out very rare tokens that are unlikely to be useful for similarity calculations or other downstream tasks.

Using `CountVectorizer` create a structured representation of the text based on the frequency of important words.

In [26]:
from pyspark.ml.feature import CountVectorizer

df_safe = tokenized_rw_nsw_nn_sample \
    .filter(col("review/text").isNotNull())

cv = CountVectorizer(
    inputCol="tokens/nostopwords/nonumbers",
    outputCol="features",
    vocabSize=5000,
    minDF=10
)

model = cv.fit(df_safe)
vocab = model.vocabulary


In [27]:
top_words = model.vocabulary[:30]
print("Top 30 tokens (approx. by frequency):")
for i, token in enumerate(top_words, start=1):
    print(f"{i:2d}. {token}")

Top 30 tokens (approx. by frequency):
 1. book
 2. read
 3. one
 4. quot
 5. story
 6. like
 7. books
 8. good
 9. time
10. great
11. reading
12. life
13. first
14. well
15. many
16. much
17. people
18. also
19. really
20. love
21. characters
22. novel
23. even
24. get
25. way
26. author
27. think
28. written
29. world
30. years


The top 30 terms in the vocaboulary are roughly represented by this way.  However, since CountVectorizer() merely approximates the actual frequency of the top words, it does not display the frequency, I'm looking for

In [28]:
from pyspark.sql.functions import explode, col

exploded = df_safe.select(explode(col("tokens/nostopwords/nonumbers")).alias("token"))

# Filtering only the top-20 tokens
topK_set = set(model.vocabulary[:20])  # top list  20 from vocabulary
filtered = exploded.filter(col("token").isin(topK_set))

exact_counts = (filtered.groupBy("token").count().orderBy(col("count").desc()))
exact_counts.show(truncate=False)

+-------+------+
|token  |count |
+-------+------+
|book   |239998|
|read   |99731 |
|one    |72419 |
|quot   |65743 |
|story  |49194 |
|like   |45138 |
|books  |36776 |
|good   |36210 |
|time   |36095 |
|great  |35893 |
|reading|33765 |
|life   |32685 |
|first  |31778 |
|well   |31100 |
|many   |30319 |
|much   |27813 |
|people |26363 |
|also   |26269 |
|really |26079 |
|love   |26017 |
+-------+------+



Calculates the exact frequency of the top 20 words from the vocabulary. It shows the actual counts.

In [29]:
threshold = 0.05 * size # % of the sample size and can be adjust
print(threshold)

7516.200000000001


Calculating a frequency threshold for tokens based on a percentage of the sample size (size).

In [30]:
highest_freq_tokens = (exact_counts.filter(col("count") > threshold).select("token")
      .rdd.flatMap(lambda x: x)
      .collect()
)


In [31]:
highest_freq_tokens

['book',
 'read',
 'one',
 'quot',
 'story',
 'like',
 'books',
 'good',
 'time',
 'great',
 'reading',
 'life',
 'first',
 'well',
 'many',
 'much',
 'people',
 'also',
 'really',
 'love']

Retrieving high-frequency token words.

#Scalability

We must filter out the entries that contain empty lists because Sparks does not function properly with them.  To accomplish this, I may use the size() and isNull() functions to determine whether the list is null or empty.

---



Filtering out stop words in the tokenised data.

In [32]:
def remove_custom_stopwords(tokens):
    return [t for t in tokens if t not in highest_freq_tokens]

remove_udf = udf(remove_custom_stopwords, ArrayType(StringType()))

tokenized_sample_2 = tokenized_rw_nsw_nn_sample.withColumn(
    "tokens_clean",
    remove_udf(col("tokens/nostopwords/nonumbers"))
)

In [33]:
from pyspark.sql.functions import size
df_safe_clean = tokenized_sample_2.filter(col("review/text").isNotNull())

empty_tokens_count = df_safe_clean.filter(
    (size(col("tokens_clean")) == 0) |
    (col("tokens_clean").isNull())
).count()

print(f"clean empty tokens: {empty_tokens_count}")

clean empty tokens: 275


Filtering rows with empty list becasue spark doesnot work well with lists.

In [34]:
cv = CountVectorizer(
    inputCol="tokens_clean",
    outputCol="features",
    vocabSize=5000,
    minDF=10
)

model2 = cv.fit(df_safe_clean)
vocab2 = model2.vocabulary


Checking for rows where the tokens_clean column contains an empty list or is null.

different top words

In [35]:
top20 = model2.vocabulary[:20]
print(top20)
for i, token in enumerate(top20, start=1):
    print(f"{i:2d}. {token}")

['characters', 'novel', 'even', 'get', 'way', 'author', 'think', 'written', 'world', 'years', 'know', 'best', 'new', 'work', 'find', 'little', 'found', 'never', 'make', 'two']
 1. characters
 2. novel
 3. even
 4. get
 5. way
 6. author
 7. think
 8. written
 9. world
10. years
11. know
12. best
13. new
14. work
15. find
16. little
17. found
18. never
19. make
20. two


Applying the CountVectorizer again, but this time to the tokens_clean column, which contains the tokens after removing stop words, numbers, and your custom high-frequency words.

Similar to the previous use of CountVectorizer, this step is building a vocabulary of the most frequent remaining tokens and will convert the text data into numerical feature vectors based on these tokens.

The parameters vocabSize=5000 and minDF=10 are used again to limit the vocabulary size to the top 5000 most frequent tokens and only include tokens that appear in at least 10 documents. This process prepares the cleaned text data for the next step, which involves converting these token counts into a format suitable for similarity calculations using HashingTF.

In [36]:
# Removing Duplicates
df_safe = df_safe_clean.dropDuplicates(["review/text"])

#Local Sensitive Hashing
is a method that hashes similar input items into the same "buckets" with high probability. It is used to find similar items in large datasets efficiently.

In [37]:
sim_threshold = 0.05 # adjust as needed


In [38]:
df_safe.show(10)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|      reviews/tokens|  tokens/nostopwords|tokens/nostopwords/nonumbers|        tokens_clean|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+
|0595229751|Tranceformers: Sh...| NULL|A3N4LVV9SE7BM8|"Shea Bower ""Dr....|             15/20|         5.0| 1159401600|           """Lights|        ..... Action|            [action]|            [action]|                    [action]|      

In [39]:

# Filtering out rows where tokens_clean is empty
df_empty_Cleantoken = df_safe.filter((df_safe.tokens_clean.isNotNull()) & (size(df_safe.tokens_clean) > 0))

In [40]:
df_empty_Cleantoken = df_empty_Cleantoken.dropDuplicates(["tokens_clean"])

In [41]:
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(inputCol="tokens_clean", outputCol="features", numFeatures= 8192)
df_featurized_Cleantoken = hashingTF.transform(df_empty_Cleantoken)

In [42]:
df_featurized_Cleantoken.show(10)

+----------+--------------------+------+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+--------------------+
|        Id|               Title| Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|      reviews/tokens|  tokens/nostopwords|tokens/nostopwords/nonumbers|        tokens_clean|            features|
+----------+--------------------+------+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+--------------------+
|B000JJVHZE|To Kill A Mocking...|  NULL|          NULL|                NULL|               2/4|         3.0| 1166486400|To Kill A Mocking...|12/19/06To Kill a...|[12, 19, 0

In [43]:
from pyspark.ml.feature import MinHashLSH

mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=10) # You can change the number of hash tables at your will
model = mh.fit(df_featurized_Cleantoken)

In [44]:
# Filtering out rows where tokens_clean is empty
df_empty_Cleantoken = df_safe.filter((df_safe.tokens_clean.isNotNull()) & (size(df_safe.tokens_clean) > 0))

# Removing duplicates based on clean tokens
df_empty_Cleantoken = df_empty_Cleantoken.dropDuplicates(["tokens_clean"])

# Featurizing the clean tokens
hashingTF = HashingTF(inputCol="tokens_clean", outputCol="features", numFeatures= 8192)
df_featurized_Cleantoken = hashingTF.transform(df_empty_Cleantoken)


similar_pairs = model.approxSimilarityJoin(
    df_featurized_Cleantoken, df_featurized_Cleantoken, threshold=sim_threshold, distCol="JaccardDistance"
).filter("datasetA.Id < datasetB.Id")  # to avoid duplicate/reverse pairs

similar_pairs = similar_pairs.orderBy("JaccardDistance", ascending=True)

Note: Jaccard Distance is the opposite of Jaccard Similarity. It is defined as 1 - Jaccard Similarity. So, the lower the Jaccard Distance, the more similar the two items are. Hence, we are ordering the pairs by Jaccard Distance in ascending order, in order to get the most similar pairs first.

In [45]:
similar_pairs.select(
    "datasetA.Title", "datasetB.Title", "datasetA.Id", "datasetB.Id", "datasetA.review/text", "datasetB.review/text", "JaccardDistance"
).show(10, truncate=False)

+------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+----------+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------